#  BioSCape SMCE Basics
## BioSCape Data Skills Workshop: From the Field to the Image
![Bioscape](images/121229-87.png)

[**BioSCape**](https://www.bioscape.io/), the Biodiversity Survey of the Cape, is NASA’s first biodiversity-focused airborne and field campaign that was conducted in South Africa in 2023. BioSCape’s primary objective is to study the structure, function, and composition of the region’s ecosystems, and how and why they are changing. 

BioSCape's airborne dataset is unprecedented, with `AVIRIS-NG`, `PRISM`, and `HyTES` imaging spectrometers capturing spectral data across the UV, visible and infrared at high resolution and `LVIS` acquiring coincident full-waveform lidar. BioSCape's `field dataset` is equally impressive, with 18 PI-led projects collecting data ranging from the diversity and phylogeny of plants, kelp and phytoplankton, eDNA, landscape acoustics, plant traits, blue carbon accounting, and more

This workshop will equip participants with the skills to find, subset, and visualize the various BioSCape field and airborne (imaging spectroscopy and full-waveform lidar) data sets. Participants will learn data skills through worked examples in terrestrial and aquatic ecosystems, including: wrangling lidar data, performing band math calculations, calculating spectral diversity metrics, machine learning and image classification, and mapping functional traits using partial least squares regression. The workshop format is a mix of expert talks and interactive coding notebooks and will be run through the BioSCape Cloud computing environment.

**Date:** October 9 - 11, 2024 Cape Town, South Africa</center>

**Host:** NASA’s Oak Ridge National Laboratory Distributed Active Archive Center (ORNL DAAC), in close collaboration with BioSCape, the South African Environmental Observation Network (SAEON), the University of Wisconsin Madison (Phil Townsend), The Nature Conservancy (Glenn Moncrieff), the University of California Merced (Erin Hestir), the University of Cape Town (Jasper Slingsby), Jet Propulsion Laboratory (Kerry Cawse-Nicholson), and UNESCO.

**Instructors:** 
- In-person contributors: Anabelle Cardoso, Erin Hestir, Phil Townsend, Henry Frye, Glenn Moncrieff, Jasper Slingsby, Michele Thornton, Rupesh Shrestha
- Virtual contributors: Kerry Cawse-Nicholson, Nico Stork, Kyle Kovach

**Audience:** This training is primarily intended for government natural resource management agency representatives and field technicians in South Africa, as well as local academics and students, especially those connected to the BioSCape Team. 

## Overview
This tutorial will explore BioSCape Science Managed Cloud Environment (SMCE) including how to access and explore using Python methods.

### Load Python Modules

In [ ]:
import s3fs
s3 = s3fs.S3FileSystem(anon=False)

## Explore the BioSCape SMCE S3 data holdings

Let's start by exploring the BioSCape Airborne data currently available on the cloud in Amazon Storage. This AWS S3 storage is specific to the SMCE that provides data access and analytics environment to the BioSCape Science Team as well as interested researchers.  
We'll learn how to mount the S3 object storage on our local environment, as well as how to bring other data to the analysis.

- **SMCE** = Science Managed Cloud Environment
- **S3** = Amazon Simple Storage Service (S3) is a cloud storage service that allows users to store and retrieve data
- **S3 Bucket** = Buckets are the basic containers that hold data. Buckets can be likened to file folders and object storage
- **S3Fs** is a `Pythonic` open source tool that mounts S3 object storage locally.  S3Fs provides a filesystem-like interface for accessing objects on S3.
    - The top-level class **`S3FileSystem`** holds connection information and allows typical file-system style operations like `ls`, `cp`, `mv`
    - `ls` is a UNIX command to list computer files and directories

In [ ]:
# Use S3Fs to list the BioSCape data on the BioSCape SMCE S3 storaage
files = s3.ls('bioscape-data/')
files

## BioSCape Flight Data

The BioSCape Campaign (Oct - Nov, 2023) flew 4 airborne instruments on 2 aircraft. 

- `Gulfstream 3`: **AVIRIS-Next Generation** and **PRISM**
- `Gulfstream 5`: **HyTES** and **LVIS**

The NASA Jet Propulsion Laboratory provides the **BioSCape Data Portal** (https://popo.jpl.nasa.gov/mmgis-aviris/?mission=BIOSCAPE) which shows flight line visualization, quick look images, and BioSCape Team Regions of Interest (ROIs).  Download access to preliminary airborne data is available.

### Let's look deeper into each airborne folder on the SMCE
#### **AVIRIS-Next Generation (AVNG)**
- We'll spend a little more time looking into the AVIRIS-NG files as these data are a focus of many Notebooks

In [ ]:
AVNG_flightlines = s3.ls('bioscape-data/AVNG/')
AVNG_flightlines[:10]

In [ ]:
AVNGfl_count = len(AVNG_flightlines)
AVNGfl_count

In [ ]:
# looking into the ang20231022t092801 folder
AVNG_scenes = s3.ls('bioscape-data/AVNG/ang20231022t092801')
AVNG_scenes

In [ ]:
# Explore the AVNG folder that holds an AVNG scene's data
AVNG_scene_data = s3.ls('bioscape-data/AVNG/ang20231022t092801/ang20231022t092801_000')
AVNG_scene_data

#### File **Naming Conventions** provide information about each flight line, scene, and data type.

- **`L1B`** are AVIRIS-NG Surface Radiance
- **`L2A`** are AVIRIS-NG Surface Reflectance

| Dataset | Description | 
| -------- | --- |
| *_RFL_ORT_QL.tif | Reflectance Quick Look Image (3 band) |
| *-RFL_ORT | Reflectance ENVI binary file (425 band)|
| *_RFL_ORT.hdr | Reflectance ENVI header file (txt file)|

![ANG_imaging](images/ANG_naming.jpg)

#### Open a file from an S3 Bucket - S3Fs
- Calling `open()` on a **`S3FileSystem`** (typically using a context manager) provides an S3File for read or write access to a particular key.
- can be used with other projects that consume the file interface like `gzip` or `pandas`.

In [ ]:
# Print the first 12 lines of and ENVI header file
hdr_link = 'bioscape-data/AVNG/ang20231022t092801/ang20231022t092801_000/ang20231022t092801_000_L2A_OE_main_27577724_RFL_ORT.hdr'
with s3.open(hdr_link, mode='r') as hdr:
    for i in range(12):
        line = next(hdr).strip()
        print(line) 

#### **Portable Remote Imaging Spectrometer (PRISM)**

In [ ]:
PRISM_flightlines = s3.ls('bioscape-data/PRISM')
PRISM_flightlines

#### There are Level 2 (L2) for PRISM data in the SCME
- ENVI file format in binary/header pairs

In [ ]:
PRISM_flightlines = s3.ls('bioscape-data/PRISM/L2')
PRISM_flightlines[:10]

- the PRISM are available in ENVI file formats as binary/header pairs

#### **Land, Vegetation, and Ice Sensor (LVIS)**

In [ ]:
LVIS_flightlines = s3.ls('bioscape-data/LVIS/')
LVIS_flightlines

#### Let's look into the LVIS Level 2 (L2) data

In [ ]:
LVIS_flightlines_L2 = s3.ls('bioscape-data/LVIS/L2')
LVIS_flightlines_L2[:10]

- LVIS L2 data are ASCII Text files

#### **Hyperspectral Thermal Emissions Spectrometer (HyTES)**
- BioSCape HyTES data is currently available from [**NASA JPL HyTES**](https://hytes.jpl.nasa.gov/order!) distribution site